In [49]:
import os
import mlflow
import yaml
import time
import optuna
import psycopg2 as psycopg
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from optuna import distributions
from optuna.integration import OptunaSearchCV
from catboost import CatBoostRegressor
from sklearn.preprocessing import FunctionTransformer

TABLE_NAME = "flat_cleaned_churn"
RANDOM_STATE = 42

In [50]:
#Создаю необходимые соединения

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] =  os.getenv("AWS_SECRET_ACCESS_KEY")

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# Установка URI для tracking и registry
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

In [51]:
#Создаю эксперимент

EXPERIMENT_NAME = "Final improved model_2"
RUN_NAME = "Nick_project_cbr_model_2"
REGISTRY_MODEL_NAME = 'CBR_model_2'

In [52]:
#Загружаю необходимые артефакты.

df = pd.DataFrame(data, columns=columns)

columns_list = df.columns.tolist()
columns_text = ",".join(columns_list) 
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(columns_text)
    
df.to_csv("flat_cleaned_churn", index=False) 


In [54]:
#Функция для предобработки данных.

def transform_data(df):
    unique_counts = df.nunique()
    columns_to_drop = unique_counts[unique_counts < 2].index
    df = df.drop(columns=columns_to_drop)
    
    # Преобразование значений 'True' и 'False' в столбцах has_elevator и is_apartment в числовой формат
    df['has_elevator'] = df['has_elevator'].str.lower().replace({'false': 0, 'true': 1}).astype(int)
    
    df.drop('id', axis=1, inplace=True)
    df.drop('build_id', axis=1, inplace=True)
    df.drop('is_apartment', axis=1, inplace=True)
    
    df['build_year'] = pd.to_datetime(df['build_year']).dt.year
    df['build_year'] = pd.to_numeric(df['build_year'], errors='coerce')
    
    return df

In [55]:
df = transform_data(df)

/tmp/ipykernel_18908/1572170628.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['has_elevator'] = df['has_elevator'].str.lower().replace({'false': 0, 'true': 1}).astype(int)


In [56]:
df.head()

,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,kitchen_area,living_area,rooms,total_area,target
0,1994,4,55.834713,37.448383,2.64,204,17,1,16,10.10,44.799999,3,73.800003,13390000.0
1,1960,1,55.701302,37.738918,2.80,20,5,0,1,6.00,16.500000,1,32.000000,5500000.0
2,1977,0,55.851589,37.416008,2.48,168,12,1,9,10.30,44.000000,3,66.000000,9500000.0
3,1970,6,55.876389,37.716415,2.64,98,9,1,5,6.50,28.000000,2,40.000000,9950000.0
4,2014,4,55.734455,37.412422,2.64,274,10,1,9,10.08,45.720001,3,75.099998,18500000.0


In [57]:
#Закладываю функцию для категоризации по этажам

def floor_category(X, **kwargs):
    new_features = pd.DataFrame(index=X.index)
    new_features['is_first_floor'] = (X['floor'] == 1).astype(int)
    new_features['is_last_floor'] = (
        (X['floor'] == X['floors_total']).astype('int')
    )
    new_features['floor_relative'] = X['floor'] / X['floors_total']
    return new_features

custom_features_transformer = FunctionTransformer(floor_category)

In [58]:
#Формирую простой препроцессов в рамках которого будут заполнены пропущенные значения и квартиры категоризируются по этажности.

preprocessor = ColumnTransformer([
     ('fill_missing',  SimpleImputer(missing_values=0., strategy='median'), ['kitchen_area', 'living_area', 'total_area']),
     ('floor_category', FunctionTransformer(floor_category), ['floor', 'floors_total'])], remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas')

In [59]:
#Формирую пайплайн

cbr_model = Pipeline([
    ('preprocessor', preprocessor),
    ('model', CatBoostRegressor(random_state=RANDOM_STATE,
                                verbose=False))
])

In [60]:
#Разделяю выборку

X = df.drop('target', axis=1).copy()
y = df['target'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [61]:
#Запускаю "голосование"

param_dist = {
    'model__iterations': distributions.IntDistribution(600, 1200),
    'model__learning_rate': distributions.FloatDistribution(.001, 0.1,
                                                            log=True),
    'model__depth': distributions.IntDistribution(4, 10),
}

study = OptunaSearchCV(
    estimator=cbr_model,
    param_distributions=param_dist,
    cv=4,
    n_jobs=-1,
    scoring='neg_mean_absolute_error',
    random_state=RANDOM_STATE,
    refit=False,
    n_trials=10
)

study.fit(X_train, y_train)

/tmp/ipykernel_18908/1998397616.py:10: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  study = OptunaSearchCV(
[I 2024-06-06 08:12:01,977] A new study created in memory with name: no-name-08ef1341-6d7a-4443-a0ea-a486f72c31a8
/home/mle-user/mle_projects/mle-project-sprint-3-v001/.mle-project-sprint-3-v001/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
[I 2024-06-06 08:12:12,416] Trial 1 finished with value: -1836312.8498902663 and parameters: {'model__iterations': 628, 'model__learning_rate': 0.059186633017457926, 'model__depth': 4}. Best is trial 1 with value: -1836312.8498902663.
/home/mle-user/mle_projects/mle-project-sprint-3-v001/.mle-project-sprint-3-v001/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` i

OptunaSearchCV(cv=4,
               estimator=Pipeline(steps=[('preprocessor',
                                          ColumnTransformer(remainder='passthrough',
                                                            transformers=[('fill_missing',
                                                                           SimpleImputer(missing_values=0.0,
                                                                                         strategy='median'),
                                                                           ['kitchen_area',
                                                                            'living_area',
                                                                            'total_area']),
                                                                          ('floor_category',
                                                                           FunctionTransformer(func=<function floor_category at 0x7eff8f168a60>),
                                                                           ['floor',
                                                                            'floors_total'])],
                                                            verbose...
                                          <catboost.core.CatBoostRegressor object at 0x7eff56cd7b80>)]),
               n_jobs=-1,
               param_distributions={'model__depth': IntDistribution(high=10, log=False, low=4, step=1),
                                    'model__iterations': IntDistribution(high=1200, log=False, low=600, step=1),
                                    'model__learning_rate': FloatDistribution(high=0.1, log=True, low=0.001, step=None)},
               random_state=42, refit=False, scoring='neg_mean_absolute_error')

In [62]:
best_params = study.best_params_
best_params

{'model__iterations': 934,
 'model__learning_rate': 0.024634376443648888,
 'model__depth': 9}

In [63]:
#обучение модели на лучших гиперпараметрах.
cbr_model.set_params(**best_params)

cbr_model.fit(X_train, y=y_train)

/home/mle-user/mle_projects/mle-project-sprint-3-v001/.mle-project-sprint-3-v001/lib/python3.10/site-packages/sklearn/compose/_column_transformer.py:1624: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('fill_missing',
                                                  SimpleImputer(missing_values=0.0,
                                                                strategy='median'),
                                                  ['kitchen_area',
                                                   'living_area',
                                                   'total_area']),
                                                 ('floor_category',
                                                  FunctionTransformer(func=<function floor_category at 0x7eff8f168a60>),
                                                  ['floor', 'floors_total'])],
                                   verbose_feature_names_out=False)),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x7eff56cd7b80>)])

In [64]:
#Функция для скоринга модели
def evaluate(pipeline, X_test, y_test):
    start_time = time.time()
    prediction = pipeline.predict(X_test)
    end_time = time.time()
    prediction_time = end_time - start_time
    mean = y_test.mean() 
    MAE = mean_absolute_error(y_test, prediction)
    MSE = mean_squared_error(y_test, prediction)
    R2 = r2_score(y_test, prediction)
    y_error = y_test - prediction
    y_error_abs = abs(y_error)
    perc_error_abs = y_error_abs / y_test
    mape = perc_error_abs.mean()
    
    return {
        'Среднее значение целевой переменной': mean.round(2),
        'Средний модуль ошибки': MAE.round(2),
        'Средняя квадратичная ошибка': MSE.round(2),
        'Коэффициент детерминации': R2,
        'Средняя абсолютная ошибка в процентах': mape.round(2),
        'Время предсказания в секундах': prediction_time
    }

In [65]:
sbr_score = evaluate(cbr_model, X_test, y_test)
sbr_score

{'Среднее значение целевой переменной': 11079136.06,
 'Средний модуль ошибки': 1799990.45,
 'Средняя квадратичная ошибка': 4987266981445.6,
 'Коэффициент детерминации': 0.6298989591550268,
 'Средняя абсолютная ошибка в процентах': 0.18,
 'Время предсказания в секундах': 0.015512704849243164}

In [66]:
#Проведу микротест чтобы понять что в дальнейшем смогу получить предсказание от модели передав набор данных.

X_valid = {
    "build_year": [1994],
    "building_type_int": ["4"],
    "latitude": [55.834713],
    "longitude": [37.448383],
    "ceiling_height": [2.64],
    "flats_count": [204],
    "floors_total": [17],
    "has_elevator": [1],
    "floor": [16],
    "kitchen_area": [10.10],
    "living_area": [44.799999],
    "rooms": [3],
    "total_area": [73.800003]
}

X_valid = pd.DataFrame(X_valid)

X_valid

,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,kitchen_area,living_area,rooms,total_area
0,1994,4,55.834713,37.448383,2.64,204,17,1,16,10.1,44.799999,3,73.800003


In [67]:
test_prediction = cbr_model.predict(X_valid)
test_prediction.round(2) 

array([16073731.11])

In [77]:
#резервно сохраняю модель с расширением bin
import joblib

# Путь для сохранения модели в формате .bin
model_path = '/home/mle-user/mle_projects/mle-project-sprint-3-v001/services/model/cbr_model.bin'

# Сохранение модели в формате .bin с помощью joblib
joblib.dump(cbr_model, model_path)

# Проверка успешного сохранения модели
import os
if os.path.exists(model_path):
    print("Модель успешно сохранена в формате .bin")
else:
    print("Ошибка при сохранении модели")

Модель успешно сохранена в формате .bin


In [69]:
#Смело логирую модель и артефакты в MLflow

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id
    
pip_requirements = '../requirements.txt' 
input_example = input_example = X_test[:10]
metadata = {'model_type': 'monthly'}

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(cbr_model,  artifact_path="models", pip_requirements=pip_requirements,
                             input_example=input_example, metadata=metadata,
                             registered_model_name=REGISTRY_MODEL_NAME, await_registration_for=6)
    # Дополнительные логирования метрик и артефактов
    mlflow.log_metrics(sbr_score)
    mlflow.log_params(best_params) 
    mlflow.log_artifact("columns.txt", artifact_path="dataframe")
    mlflow.log_artifact("flat_cleaned_churn", artifact_path="dataframe")

/home/mle-user/mle_projects/mle-project-sprint-3-v001/.mle-project-sprint-3-v001/lib/python3.10/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/06/06 08:14:26 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (curre

In [70]:
#Проверяю доступность модели.
model_uri = "models:/CBR_model_2/2"  # Укажите URI вашей модели
loaded_model = mlflow.sklearn.load_model(model_uri)

In [73]:
# Сохраняю модель в директории /services/model
target_dir = "/home/mle-user/mle_projects/mle-project-sprint-3-v001/services/model"

os.makedirs(target_dir, exist_ok=True)

mlflow.sklearn.save_model(loaded_model, target_dir)

print("Модель сохранена в директории:", target_dir)

Модель сохранена в директории: /home/mle-user/mle_projects/mle-project-sprint-3-v001/services/model


In [12]:
#проверяю возможность загрузки модели
import joblib

def load_churn_model(model_path: str):
    try:
        model = joblib.load(model_path)  # Загрузка модели с использованием joblib
        print("Model loaded successfully")
    except Exception as e:
        print(f"Failed to load model: {e}")
        model = None
    return model

if __name__ == "__main__":
    model_path = "../services/model/model.pkl"
    model = load_churn_model(model_path)


Model loaded successfully
